<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/noisy_queries_typo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga das typos do Wikipedia

In [33]:
import requests

req = requests.get('https://en.wikipedia.org/wiki/Wikipedia:Lists_of_common_misspellings/For_machines')
req

<Response [200]>

In [34]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text, 'html.parser')
misspellings = soup.find('pre').text.splitlines()
for line in misspellings[:5]:
    print(line)

abandonned->abandoned
aberation->aberration
abilityes->abilities
abilties->abilities
abilty->ability


In [68]:
typo = {}
q = 0

def add_misspelling(misspelled, correction, q):
    correction = correction.strip()
    misspelled = misspelled.strip()
    if correction not in typo:
        typo[correction] = []
    typo[correction].append(misspelled)
    return q + 1
    
for line in misspellings:
    misspelled, correction = line.split('->')
    if ',' in correction:
        for correction in correction.split(','):
            q = add_misspelling(misspelled, correction, q)
    else:
        q = add_misspelling(misspelled, correction, q)

print ('words:', len(typo), '   typos:', q)        

words: 3265    typos: 4528


## Carga das queries originais

In [36]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/queries-originals.txt

--2021-11-19 17:58:25--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/queries-originals.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2275 (2.2K) [text/plain]
Saving to: ‘queries-originals.txt’

queries-originals.t 100%[===================>]   2.22K  --.-KB/s    in 0s      

2021-11-19 17:58:25 (43.4 MB/s) - ‘queries-originals.txt’ saved [2275/2275]



In [55]:
import string 

originals = []
bags = []

for line in open('queries-originals.txt'):
    bags.append(line.strip().translate(str.maketrans('', '', string.punctuation)).split()[1:])
    fields = line.strip().split()
    originals.append((int(fields[0]), ' '.join(fields[1:])))

In [59]:
originals[-5:]

[(1132532, 'average annual income data analyst'),
 (1133579, 'how does granulation tissue start'),
 (1136043, 'difference between a hotel and motel'),
 (1136047, "difference between a company's strategy and business model is"),
 (1136962,
  'why did the ancient egyptians call their land kemet, or black land?')]

In [60]:
bags[-5:]

[['average', 'annual', 'income', 'data', 'analyst'],
 ['how', 'does', 'granulation', 'tissue', 'start'],
 ['difference', 'between', 'a', 'hotel', 'and', 'motel'],
 ['difference',
  'between',
  'a',
  'companys',
  'strategy',
  'and',
  'business',
  'model',
  'is'],
 ['why',
  'did',
  'the',
  'ancient',
  'egyptians',
  'call',
  'their',
  'land',
  'kemet',
  'or',
  'black',
  'land']]

# Produção dos typos a partir da Wikipedia

In [194]:
import random

noisy = {}

random.seed(1234)

missing = []
for i, bag in enumerate(bags):
    options = []
    for word in bag:
        if word in typo:
            options.append(word)
    if options:
        original = random.choice(options)
        misspelled = random.choice(typo[original])
        noisy[str(originals[i][0])] = originals[i][1].replace(original, misspelled, 1)
    else:
        missing.append(i)

noisy

{'1064670': 'why do hunters pattern ther shotguns?',
 '1071750': 'why is pete rised banned from hall of fame',
 '1105792': 'deffine: geon',
 '1106979': 'deffine pareto chart in statistics',
 '1108651': 'waht the best way to get clothes white',
 '1109707': 'what medium do radio waves travel throught',
 '1110678': 'what is tjhe un fao',
 '1113256': "whta is reba mcentire's net worth",
 '1115210': 'whta is chaff and flare',
 '1116380': 'what is a nonconformity? earth scinece',
 '1121353': 'what can you do boaut discrimination in the workplace in oklahoma city',
 '1122767': 'waht amino produces carnitine',
 '1127540': 'meaing of shebang',
 '1132532': 'average anual income data analyst',
 '1133579': 'how does granulation tissue strat',
 '1136043': 'difference betwen a hotel and motel',
 '1136047': "difference between a company's strategy and busness model is",
 '1136962': 'why did thge ancient egyptians call their land kemet, or black land?',
 '118440': 'deffine bmt medical',
 '121171': 'de

# 9 Restantes - Produção manual dos typos a partir do Dumbtionary ou livremente

http://www.dumbtionary.com/ - Search over misspelled words

In [195]:
[originals[i] for i in missing]

[(47210, 'average wedding dress alteration cost'),
 (156498, 'do google docs auto save'),
 (336901, 'how old is vanessa redgrave'),
 (405163, 'is caffeine an narcotic'),
 (1030303, 'who is aziz hashim'),
 (1037496, 'who is rep scalise?'),
 (1043135, 'who killed nicholas ii of russia'),
 (1051399, 'who sings monk theme song'),
 (1131069, 'how many sons robert kraft has')]

In [196]:
noisy['47210'] = 'average weddin dress alteration cost' #dumbtionary
noisy['156498'] = 'do google docs alto save'
noisy['336901'] = 'how old is Vanesa redgrave' #dumbtionary - adaptado
noisy['405163'] = 'is caffein an narcotic' #dumbtionary
noisy['1030303'] = 'whom is aziz hashim'
noisy['1037496'] = 'who is rep scaliz?'
noisy['1043135'] = 'who killd nicholas ii of russia'
noisy['1051399'] = 'who sings mank theme song'
noisy['1131069'] = 'how many sons robert kraft have'
noisy

{'1030303': 'whom is aziz hashim',
 '1037496': 'who is rep scaliz?',
 '1043135': 'who killd nicholas ii of russia',
 '1051399': 'who sings mank theme song',
 '1064670': 'why do hunters pattern ther shotguns?',
 '1071750': 'why is pete rised banned from hall of fame',
 '1105792': 'deffine: geon',
 '1106979': 'deffine pareto chart in statistics',
 '1108651': 'waht the best way to get clothes white',
 '1109707': 'what medium do radio waves travel throught',
 '1110678': 'what is tjhe un fao',
 '1113256': "whta is reba mcentire's net worth",
 '1115210': 'whta is chaff and flare',
 '1116380': 'what is a nonconformity? earth scinece',
 '1121353': 'what can you do boaut discrimination in the workplace in oklahoma city',
 '1122767': 'waht amino produces carnitine',
 '1127540': 'meaing of shebang',
 '1131069': 'how many sons robert kraft have',
 '1132532': 'average anual income data analyst',
 '1133579': 'how does granulation tissue strat',
 '1136043': 'difference betwen a hotel and motel',
 '11

In [208]:
with open('noisy-queries-typo.txt', 'w') as w:
    for id in noisy.keys():
        w.write(str(id) + ' ' + noisy[id] + '\n')

# Composição do arquivo mestre de queries ruidosas

In [197]:
import pandas as pd

df = pd.DataFrame([(str(number), text) for number, text in originals], columns=['Query#', 'original'])
df.set_index('Query#', inplace=True)
df

,original
Query#,
23849,are naturalization records public information
42255,average salary for dental hygienist in nebraska
47210,average wedding dress alteration cost
67316,can fever cause miscarriage early pregnancy
118440,define bmt medical
121171,define etruscans
135802,definition of laudable
141630,describe how muscles and bones work together t...
156498,do google docs auto save


In [198]:
df = df.join(pd.DataFrame.from_dict(noisy, orient='index', columns=['noisy-typo']))
df

,original,noisy-typo
Query#,,
23849,are naturalization records public information,are naturalization records public infomation
42255,average salary for dental hygienist in nebraska,average salery for dental hygienist in nebraska
47210,average wedding dress alteration cost,average weddin dress alteration cost
67316,can fever cause miscarriage early pregnancy,can fever casue miscarriage early pregnancy
118440,define bmt medical,deffine bmt medical
121171,define etruscans,deffine etruscans
135802,definition of laudable,definiton of laudable
141630,describe how muscles and bones work together t...,describe how muscles adn bones work together t...
156498,do google docs auto save,do google docs alto save


In [201]:
import requests

req = requests.get('https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/noisy-queries.csv')
if req.status_code != 200:
    df_mestre = df[['original', 'noisy-typo']]

In [205]:
import csv
import numpy as np

if req.status_code == 200:
    query_list = list(csv.reader(req.content.decode('utf-8').splitlines(), delimiter=','))
    df_mestre = pd.DataFrame(query_list[1:], columns = query_list[0], dtype=int).set_index('Query#').join(df[['noisy-typo']])

In [206]:
df_mestre

,original,noisy-translation-pt,noisy-typo
Query#,,,
23849,are naturalization records public information,is public information from naturalization reco...,are naturalization records public infomation
42255,average salary for dental hygienist in nebraska,Average Salary of Dental Hygienist in Nebraska,average salery for dental hygienist in nebraska
47210,average wedding dress alteration cost,average cost of changing the wedding dress,average weddin dress alteration cost
67316,can fever cause miscarriage early pregnancy,can fever cause miscarriage in early pregnancy,can fever casue miscarriage early pregnancy
118440,define bmt medical,define bmt medical,deffine bmt medical
121171,define etruscans,define Etruscans,deffine etruscans
135802,definition of laudable,definition of commendable,definiton of laudable
141630,describe how muscles and bones work together t...,describe how muscles and bones work together t...,describe how muscles adn bones work together t...
156498,do google docs auto save,Google docs automatically saves,do google docs alto save


In [207]:
df_mestre.to_csv('noisy-queries.csv', header=True, index=True, index_label='Query#', quoting = csv.QUOTE_NONNUMERIC)